# Loading Data from AWS S3

In [1]:
import boto3, os
import pandas as pd
import numpy as np

In [ ]:
access_info = pd.read_csv('rootkey.csv', header=None).values.reshape((2,))
access_key = access_info[0].split('=')[-1]
secret_key = access_info[1].split('=')[-1]

session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)

s3 = session.resource('s3')

In [ ]:
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
s3.Bucket('lish-moa').download_file('train_features.csv', 'train_features.csv')
s3.Bucket('lish-moa').download_file('train_drug.csv', 'train_drug.csv')
s3.Bucket('lish-moa').download_file('train_targets_scored.csv', 'train_targets_scored.csv')

# Reading into DataFrames

In [2]:
X = pd.read_csv('train_features.csv', index_col=0)
y = pd.read_csv('train_targets_scored.csv', index_col=0)
drugs = pd.read_csv('train_drug.csv')
X.shape, y.shape

((23814, 875), (23814, 206))

In [3]:
X.head(5)

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
drugs.head(5)

,sig_id,drug_id
0,id_000644bb2,b68db1d53
1,id_000779bfc,df89a8e5a
2,id_000a6266a,18bb41b2c
3,id_0015fd391,8c7f86626
4,id_001626bd3,7cbed3131


In [5]:
print(X['cp_type'].unique())
print(X['cp_time'].unique())
print(X['cp_dose'].unique())

['trt_cp' 'ctl_vehicle']
[24 72 48]
['D1' 'D2']


In [6]:
X = X.replace({
    'cp_type': {'trt_cp': -1.0, 'ctl_vehicle': 1.0},
    'cp_time': {24: -1.0, 48: 0.0, 72: 1.0},
    'cp_dose': {'D1': -1.0, 'D2': 1.0}
})

In [7]:
X.head(3)

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,-1.0,-1.0,-1.0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,-1.0,1.0,-1.0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,-1.0,0.0,-1.0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931


In [8]:
drugs['drug_id'].unique().shape[0]

3289

Out of 23814 samples, there are 3289 unique drugs. That means that, in our train-test splits or cross-validation folds, we need to be wary of the distribution of drug classes.

In [9]:
unique_value_counts = drugs['drug_id'].value_counts().unique()
unique_value_counts

array([1866,  718,  246,  203,  202,  196,  194,  186,  178,   19,   18,
         14,   13,   12,   11,    8,    7,    6,    5,    4,    3,    2,
          1])

In [10]:
drugs['drug_id'].value_counts()[:10]  # Top 10 most common drugs

cacb2b860    1866
87d714366     718
9f80f3f77     246
8b87a7a83     203
5628cb3ee     202
d08af5d4b     196
292ab2c28     194
d50f18348     186
d1b47f29d     178
67c879e79      19
Name: drug_id, dtype: int64

A  majority of drugs appear 6 times in the data set (shown below). However, there's one drug that is used in 1866 of the time.

In [11]:
vcs = drugs['drug_id'].value_counts()
counts = []
for n in unique_value_counts:
    counts.append(drugs[drugs['drug_id'].isin(vcs.index[vcs == n])]['drug_id'].unique().shape[0])
distribution = pd.DataFrame({'n': unique_value_counts, 'count': counts})
distribution

,n,count
0,1866,1
1,718,1
2,246,1
3,203,1
4,202,1
5,196,1
6,194,1
7,186,1
8,178,1
9,19,1


# Mutual Information Feature Subspacing

In [12]:
test_label = y.iloc(axis=1)[0]
test_label

sig_id
id_000644bb2    0
id_000779bfc    0
id_000a6266a    0
id_0015fd391    0
id_001626bd3    0
               ..
id_fffb1ceed    0
id_fffb70c0c    0
id_fffc1c3f4    0
id_fffcb9e7c    0
id_ffffdd77b    0
Name: 5-alpha_reductase_inhibitor, Length: 23814, dtype: int64

In [13]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler

In [14]:
X_scaled = StandardScaler().fit_transform(X.values)
mi = mutual_info_classif(X_scaled, test_label.values)
mi

array([1.74292078e-03, 1.65992775e-03, 2.63048818e-03, 4.89649774e-04,
       1.18230574e-04, 8.18367263e-05, 1.46986146e-04, 2.40857684e-04,
       0.00000000e+00, 2.38884686e-04, 2.36918466e-04, 2.16213844e-04,
       0.00000000e+00, 3.30137781e-05, 2.30295845e-04, 7.92049437e-05,
       0.00000000e+00, 1.36956291e-04, 0.00000000e+00, 0.00000000e+00,
       6.28896574e-05, 0.00000000e+00, 4.50733023e-04, 0.00000000e+00,
       3.34818713e-04, 3.65837787e-07, 2.29254636e-04, 3.44534289e-05,
       0.00000000e+00, 1.90162896e-04, 0.00000000e+00, 1.73063209e-04,
       5.00196132e-04, 0.00000000e+00, 1.83927742e-04, 9.82649899e-06,
       1.35010480e-04, 2.41746694e-04, 8.25023758e-05, 8.13398002e-05,
       3.22252185e-05, 3.93502142e-04, 1.80571858e-04, 0.00000000e+00,
       2.53819790e-05, 1.11937847e-04, 3.33392214e-04, 0.00000000e+00,
       0.00000000e+00, 1.00858131e-04, 7.52065860e-05, 4.96342088e-04,
       0.00000000e+00, 8.08826939e-05, 3.49016870e-05, 6.07752816e-04,
      

In [15]:
# Feature subspace by thresholding Mutual Information
v1 = StandardScaler().fit_transform(mi.reshape((-1, 1))).reshape((-1, )) > 0.95
v1

array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False,  True, False, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,

In [16]:
print("%d bytes" % (v1.size * v1.itemsize))

875 bytes


# Multilabel Stratified KFolds

Following the approach described [here](https://www.kaggle.com/cdeotte/rapids-genetic-algorithm-knn-cv-0-01840), we implement Multilabel Stratified KFold by utilizing the `iterative-stratification` library ([GitHub](https://github.com/trent-b/iterative-stratification)):

> `iterative-stratification` is a project that provides scikit-learn compatible cross validators with stratification for multilabel data. Presently `scikit-learn` provides several cross validators with stratification. However, these cross validators do not offer the ability to stratify multilabel data. This `iterative-stratification` project offers implementations of `MultilabelStratifiedKFold`, `MultilabelRepeatedStratifiedKFold`, and `MultilabelStratifiedShuffleSplit` with a base algorithm for stratifying multilabel data.

In [18]:
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

def strat_kfold(scored, drugs, folds=5, random_state=42):
    
    targets = scored.columns[1:]
    scored_temp = scored.merge(drugs, on='sig_id', how='left')
    vc = scored_temp['drug_id'].value_counts()
    vc1 = vc.loc[vc <= 18].index.sort_values()  # drugs with n <= 18
    vc2 = vc.loc[vc > 18].index.sort_values()  # drugs with n > 18
    
    drug_count_1 = {}; drug_count_2 = {};
    
    # Stratify n <= 18 
    ml_skf = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=random_state)
    temp = scored_temp.groupby('drug_id')[targets].mean().loc[vc1]
    
    for fold, (idxT, idxV) in enumerate(ml_skf.split(temp, temp[targets])):
        dd = {k:fold for k in temp.index[idxV].values}
        drug_count_1.update(dd)

    # Stratify n > 18 
    ml_skf = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=random_state)
    temp = scored_temp.loc[scored_temp['drug_id'].isin(vc2)].reset_index(drop=True)
    print(temp)
    print(temp[targets])
    for fold, (idxT, idxV) in enumerate(ml_skf.split(temp, temp[targets])):
        dd = {k:fold for k in temp['sig_id'][idxV].values}
        drug_count_2.update(dd)
    
    scored_temp['fold'] = np.nan
    scored_temp['fold'] = scored_temp['drug_id'].map(drug_count_1)
    scored_temp.loc[scored_temp['fold'].isna(), 'fold'] = scored_temp.loc[scored_temp['fold'].isna(), 'sig_id'].map(drug_count_2)
    
    return scored_temp[['sig_id', 'fold']]

In [19]:
N_FOLDS = 5
RANDOM_STATE = 42
folds = strat_kfold(y, drugs, folds=N_FOLDS, random_state=RANDOM_STATE)
folds

/home/howardwong/anaconda3/envs/rapids-0.16/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/home/howardwong/anaconda3/envs/rapids-0.16/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


            sig_id  5-alpha_reductase_inhibitor  11-beta-hsd1_inhibitor  \
0     id_0020d0484                            0                       0   
1     id_002fb9c19                            0                       0   
2     id_0054388ec                            0                       0   
3     id_0079af0fb                            0                       0   
4     id_0079d45d3                            0                       0   
...            ...                          ...                     ...   
4003  id_ffd26f361                            0                       0   
4004  id_fff3976bd                            0                       0   
4005  id_fff6df1c5                            0                       0   
4006  id_fffc1c3f4                            0                       0   
4007  id_fffcb9e7c                            0                       0   

      acat_inhibitor  acetylcholine_receptor_agonist  \
0                  0                       

,sig_id,fold
0,id_000644bb2,1.0
1,id_000779bfc,1.0
2,id_000a6266a,2.0
3,id_0015fd391,0.0
4,id_001626bd3,0.0
...,...,...
23809,id_fffb1ceed,0.0
23810,id_fffb70c0c,4.0
23811,id_fffc1c3f4,3.0
23812,id_fffcb9e7c,4.0


In [157]:
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss

oof = np.zeros((X_scaled.shape[0], 206))
               
for fold in range(N_FOLDS):
    model = KNeighborsClassifier(n_neighbors=1000)
    model.fit(X_scaled[folds[folds['fold'] != fold]['sig_id'].index][:, v1], y[y.index.isin(folds[folds['fold'] != fold]['sig_id'])])
    pp = model.predict_proba(X_scaled[folds[folds['fold'] == fold]['sig_id'].index][:, v1])
    pp = np.stack( [(1 - pp[x][:,0]) for x in range(len(pp))] ).T
    oof[folds[folds['fold'] == fold]['sig_id'].index, ] = pp

In [160]:
log_loss(y.values, oof)

3.1834516193192397

In [166]:
y_indices = list(range(len(y)))